<a href="https://colab.research.google.com/github/karsarobert/NLP2025/blob/main/04/NLP2024_04_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modellek finomhangolása

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
# PyTorch és Transformers könyvtárak importálása
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Előre definiált modell nevének kiválasztása
checkpoint = "bert-base-uncased"

# Tokenizáló létrehozása az előre definiált modell alapján
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Modell inicializálása a prediktáláshoz és finomhangoláshoz az előre definiált modell alapján
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# Input szekvenciák előkészítése a tokenizáló segítségével
sequences = [
    "I've been waiting for a NLP course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# Címkék hozzáadása a batch-hez (0 vagy 1 az adott inputra)
batch["labels"] = torch.tensor([1, 1])

# Optimizer inicializálása az AdamW segítségével a modell paramétereivel
optimizer = AdamW(model.parameters())

# A veszteség kiszámítása és visszaterjesztése a hálózaton
loss = model(**batch).loss
loss.backward()

# A gradiensek alapján az optimalizáló lépés végrehajtása
optimizer.step()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


**Természetesen a modellnek mindössze két mondaton való edzése nem fog túl jó eredményeket hozni. Ahhoz, hogy jobb eredményeket érj el, nagyobb adathalmazt kell összeállítanod.**

**Ebben a részben példaként az MRPC (Microsoft Research Paraphrase Corpus) adathalmazt fogjuk használni, amelyet William B. Dolan és Chris Brockett egy tanulmányukban mutattak be. Az adathalmaz 5801 mondatpárt tartalmaz, egy jelzéssel, ami azt mutatja, hogy parafrázisok-e vagy sem (azaz, hogy mindkét mondat ugyanazt jelenti-e).**


**A Hub nem csak modelleket tartalmaz; rengeteg adathalmazt is megtalálsz itt, sok különböző nyelven. Böngészhetsz az adathalmazok között itt: [https://huggingface.co/datasets](https://huggingface.co/datasets), és azt javasoljuk, hogy próbálj meg betölteni és feldolgozni egy új adathalmazt, miután átnézted ezt a részt (lásd az általános dokumentációt itt: [https://huggingface.co/docs](https://huggingface.co/docs)).  De most egyelőre koncentráljunk az MRPC adathalmazra! Ez a 10 adathalmaz egyike, amely a GLUE benchmarkot alkotja. Ez egy olyan tudományos mérce, amely arra szolgál, hogy megmérje az ML-modellek teljesítményét 10 különböző szövegosztályozási feladatban.**

**A 🤗 Datasets könyvtár egy nagyon egyszerű parancsot biztosít a Hubon lévő adathalmazok letöltésére és gyorsítótárazására. Az MRPC adathalmazt az alábbiak szerint tölthetjük le:**


In [ ]:
# A `datasets` könyvtárból importáljuk a `load_dataset` függvényt, amely lehetővé teszi adatkészletek betöltését.
from datasets import load_dataset

# "glue" adatkészletből a "mrpc" alcsoport betöltése.
raw_datasets = load_dataset("glue", "mrpc")

# A betöltött adatkészlet visszaadása.
raw_datasets


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

**Ahogy láthatod, egy DatasetDict objektumot kapunk, ami a tréninghalmazt, a validációs halmazt és a teszthalmazt tartalmazza. Ezek mindegyike több oszlopot foglal magában (sentence1, sentence2, label és idx), valamint változó számú sort, melyek az egyes halmazokban található elemek számai (tehát 3668 pár mondat van a tréninghalmazban, 408 a validációs halmazban és 1725 a teszthalmazban).**

**A letöltött és tárolt adathalmazt alapértelmezés szerint a ~/.cache/huggingface/datasets helyen találhatjuk meg.  A tároló mappádat a HF_HOME környezeti változó beállításával testre szabhatod.**

**A raw_dataset objektumunkban lévő mondatpárokat indexeléssel érhetjük el, akárcsak egy szótár (dictionary) esetében:**


In [ ]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

Láthatjuk, hogy a címkék már egész számok, így itt nem kell előfeldolgozást végeznünk. Ahhoz, hogy megtudjuk, melyik egész szám melyik címkének felel meg, megvizsgálhatjuk a raw_train_datasetünk jellemzőit. Ebből megtudjuk, hogy az egyes oszlopok milyen típusúak:

In [ ]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

A színfalak mögött a címke ClassLabel típusú, és az egész számok címkenévhez való hozzárendelése a names mappában van tárolva. A 0 megfelel a not_equivalentnek, az 1 pedig az equivalentnek.

In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

Azonban nem adhatunk át két szekvenciát a modellnek, és nem kaphatunk előrejelzést arról, hogy a két mondat parafrázis-e vagy sem. A két szekvenciát párként kell kezelnünk, és megfelelő előfeldolgozást kell alkalmaznunk. Szerencsére a tokenizáló is képes egy szekvenciapárt fogadni, és úgy előkészíteni, ahogy a BERT-modellünk elvárja:

In [ ]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

Láthatjuk tehát, hogy a modell a bemeneteket [CLS] mondat1 [SEP] mondat2 [SEP] formában várja el, ha két mondat van. Ezt a token_type_ids értékkel összehangolva megkapjuk:

**Ahogy láthatod, a bemenet azon részei, amelyek a [CLS] sentence1 [SEP] -nek felelnek meg, mind 0-s típus azonosítóval (token type ID) rendelkeznek, míg a többi, a sentence2 [SEP] -nek megfelelő részek mind 1-es típussal vannak ellátva.**

**Fontos megjegyezni, hogy ha más ellenőrzőpontot (checkpoin-ot) választasz, nem feltétlenül kapsz token_type_ids-t a tokenizált bemenetekben (például, ha egy DistilBERT modellt használsz, azok nem kerülnek visszaadásra). Csak akkor lesznek visszaadva, ha a modell tudni fogja, mit kezdjen velük, mivel az előképzés során már találkozott velük.**

**A BERT itt token típusa ID-kkel kerül előképzésre és a már tárgyalt maszkolt nyelvi modellezési cél mellett van egy kiegészítő feladata, az úgynevezett "következő mondat előrejelzése" (next sentence prediction). Ennek a feladatnak az a célja, hogy modellezze a mondatpárok közötti kapcsolatot.**

**A következő mondat előrejelzésekor a modell kap egy pár mondatot (véletlenszerűen maszkolt tokenekkel) és meg kell jósolnia, hogy a második mondat követi-e az elsőt. A feladat nehezítése érdekében az esetek felében a mondatok egymás után következnek az eredeti dokumentumban, amelyből kinyerésre kerültek, másik felében pedig két teljesen különböző dokumentumból származnak.**

**Általánosságban nem kell aggódnod, hogy vannak-e token_type_ids-k a tokenizált bemenetedben: amíg ugyanazt az ellenőrzőpontot használod a tokenizer és a modell esetében, minden rendben lesz, mivel a tokenizer tudja, mit kell biztosítani a modelljének.**

**Mivel láttuk, hogyan tud a tokenizerünk egy pár mondattal foglalkozni, használhatjuk a teljes adathalmaz tokenizálására: az előző gyakorlathoz hasonlóan a tokenizernek átadhatunk egy listát a mondatpárokból, úgy, hogy megadjuk az első mondatok, majd a második mondatok listáját. Ez kompatibilis a  kitöltési (padding) és csonkítási (truncation) beállításokkal is. Tehát az egyik módja a tréning adathalmaz előfeldolgozásának:**


In [ ]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

**Ez a megoldás jól működik, de van egy hátránya:  egy szótárat ad vissza (a kulcsainkkal: input_ids, attention_mask és token_type_ids; és azokkal az értékekkel, amik listák listái). Valamint csak akkor fog rendesen működni, ha elég RAM-mal rendelkezel ahhoz, hogy a teljes adathalmazon elvégezd a tokenizációt (míg a 🤗 Datasets könyvtárból származó adathalmazok Apache Arrow fájlok, a lemezen tárolva, így csak a memóriába töltöd azokat a mintákat, amelyeket lekérdezel).**

**Ahhoz, hogy az adatot adathalmaz formában tartsuk, a Dataset.map() metódust fogjuk használni. Ez lehetőséget ad extra rugalmasságra is, ha nem csak a tokenizációnál van szükségünk további előfeldolgozásra. A map() metódus úgy működik, hogy minden elemre alkalmaz egy adott függvényt, szóval alkossunk egy olyan függvényt, ami tokenizálja a bemeneteinket:**


In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

**Ez a függvény egy szótárat vesz fel (például az adathalmazunk elemeit) és egy új szótárt ad vissza input_ids, attention_mask és token_type_ids kulcsokkal. Fontos megjegyezni, hogy akkor is működik, ha az 'example' szótár több mintát tartalmaz (mindegyik kulcs egy mondatlista), hiszen a tokenizer képes mondatpárok listáján dolgozni, ahogyan azt korábban láttuk. Ez lehetővé teszi számunkra a batched=True opció használatát a map() hívásunkban, ami jelentős mértékben felgyorsítja a tokenizációt. A tokenizer mögött a 🤗Tokenizers könyvtárból való, Rust nyelven írt tokenizer működik. Ez a tokenizer nagyon gyors lehet, de csak akkor, ha nagy mennyiségű bemenetet kap egyszerre.**

**Megjegyzés: szándékosan most nem használtuk a padding  argumentumot a tokenizációs függvényünkben. Ennek az az oka, hogy nem hatékony minden mintát a maximális hosszúságra kitölteni (padding): sokkal jobb a mintákat akkor kitölteni, amikor egy köteget (batch) építünk, hiszen így arra elegendő csak a maximális hosszúságra kitölteni, és nem a teljes adathalmazéra. Ez rengeteg időt és számítási kapacitást spórolhat meg, amikor a bemenetek nagymértékben változnak hosszúságban!**

Így alkalmazhatod a tokenizációs függvényt minden adathalmazodra egyszerre. A  batched=True-t használjuk a map hívásban, így a függvény a dataset-ünk több elemére lesz egyszerre alkalmazva, így  nem minden elemre külön-külön. Ez gyorsabb előfeldolgozást tesz lehetővé.

Amikor a map() függvényt használjuk, általában egy függvényt adunk meg paraméterként, amelyet szeretnénk alkalmazni az iterálható objektum elemeire. Ezt követően azon iterálható objektumokat adjuk meg, amelyeken a függvényt alkalmazni szeretnénk.


In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

**A map() használatakor akár többprocesszos (multiprocessing) megoldást is használhatsz az előfeldolgozási funkciódhoz a num_proc argumentum megadásával. Most nem tettük ezt, mert a 🤗 Tokenizers könyvtár már több szálat használ a gyorsabb tokenizáció érdekében, ha viszont nem egy ilyen gyors tokenizerrel dolgozol, akkor ez felgyorsíthatja az előfeldolgozási folyamatot.**

A tokenize_function függvényünk egy szótárt ad eredményül, amiben szerepelnek az input_ids, attention_mask és token_type_ids kulcsok. Ennek köszönhetően mindhárom mezőt minden adathalmazunk minden részhalmazához hozzáadjuk. Fontos, hogy létező mezőket is tudnánk módosítani, ha az előfeldolgozó függvényünk egy új értéket adna eredményül arra a kulcsra, amely már létezik abban az adathalmazban, amire a map()-ot alkalmazzuk.

**Az utolsó teendőnk annyi, hogy minden példányt a leghosszabb elem hosszára töltsünk ki, amikor ezeket az elemeket kötegekbe (batch-ekbe) gyűjtjük. Ezt a technikát dinamikus kitöltésnek (dynamic padding) nevezzük.**


In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

**Az elemek köteggé (batch) alakításáért a collate függvény a felelős. Ez egy olyan paraméter, amit átadhatsz a DataLoader létrehozásakor. Az alapértelmezett működés egyszerűen PyTorch tenzorokká alakítja a mintákat és összefűzi őket (rekurzívan, ha az elemeid listák, tuple-ök vagy szótárak). Ez nem lesz megfelelő a mi esetünkben, mivel a bemeneteink különböző hosszúságúak lesznek. Szándékosan elhalasztottuk a kitöltést (padding), hogy csak akkor végezzük, ha valóban szükséges az adott kötegen, ezzel elkerüljük a túl hosszú, rengeteg paddinget tartalmazó bemeneteket. Ez felgyorsítja a tréninget, de a TPU használatakor problémákat okozhat, mivel a TPU-k a fix formát kedvelik, még akkor is, ha ez extra kitöltést igényel.**

**Gyakorlatban ehhez definiálnunk kell egy collate függvényt, ami a megfelelő mennyiségű paddinget alkalmazza az egy kötegbe kerülő elemekre. Szerencsére a 🤗 Transformers könyvtár biztosít ilyen funkciót, ez a DataCollatorWithPadding. Létrehozásakor felvesz egy tokenezőt (hogy tudja melyik padding token-t használja, valamint, hogy a modell a bemenetek melyik oldalán várja a paddinget) és minden mást elintéz, amire szükségünk van:**


In [ ]:
# Az első 8 minta kiválasztása a "train" részből
samples = tokenized_datasets["train"][:8]

# Az "idx", "sentence1" és "sentence2" kulcsok eltávolítása a mintákból
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}

# Az input tokenek hosszának listába gyűjtése az összes maradék mintára
lengths = [len(x) for x in samples["input_ids"]]


Nem meglepő, hogy különböző hosszúságú mintákat kapunk, 32-től 67-ig. A dinamikus kitöltés azt jelenti, hogy a tételben lévő mintákat mind 67 hosszúságúra kell kitölteni, ami a tételen belüli maximális hossz. Dinamikus kitöltés nélkül az összes mintát a teljes adathalmaz maximális hosszára kellene kitölteni, vagy a modell által elfogadható maximális hosszra. Ellenőrizzük le, hogy a data_collatorunk megfelelően dinamikusan kitölti a köteget:

In [ ]:
# Adatösszeállító (data collator) függvény alkalmazása a mintákra
batch = data_collator(samples)

# Az összeállított batch elemeinek alakjának lekérése és tárolása egy szótárban
{k: v.shape for k, v in batch.items()}


{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

**A Transformers egy Trainer osztállyal segít előkészített modelljeid finomhangolásában a saját adathalmazodon. A legnehezebb rész vélhetően annak a környezetnek az előkészítése lesz, ahol a Trainer.train() lefut, mivel CPU-n rendkívül lassan fog menni. Ha nincs GPU-d, ingyenes GPU-hoz vagy TPU hozzáférést szerezhetsz a Google Colab.**



##Képzés
Az első lépés, mielőtt definiálhatnánk a trénerünket, egy TrainingArguments osztály definiálása, amely tartalmazza az összes hiperparamétert, amelyet a tréner a képzéshez és a kiértékeléshez használni fog. Az egyetlen argumentum, amit meg kell adnunk, az egy könyvtár, ahová a betanított modellt elmentjük, valamint az ellenőrző pontok az út mentén. Az összes többihez meghagyhatja az alapértelmezetteket, amelyeknek elég jól kell működniük egy alapvető finomhangoláshoz.

In [ ]:
!pip install accelerate

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Látni fogod, hogy az előző órával ellentétben egy figyelmeztetést kapsz az előretrénelt modell példányosítása után.  Azért van ez, mert a BERT-et nem mondatpárok osztályozására képezték elő, tehát az előtrénelt modell fejrésze (head) el lett dobva, helyette egy új, szekvenciák osztályozására alkalmas fej került beillesztésre. A figyelmeztetés jelzi, hogy egyes súlyok (weights) nem kerültek felhasználásra (az eltávolított előtrénelési fejnek megfelelőek), illetve mások véletlenszerűen lettek inicializálva (az új fejhez tartozók). Végül arra bíztat, hogy tanítsuk be a modellt, ami pontosan a következő lépésünk lesz.**  

Miután megvan a modellünk, definiálhatjuk a Trainer-t a következők átadásával: a modell, a training_args, a tréning és validációs adathalmazok, az adat betöltő-nk (data_collator) és a tokenizer-ünk.


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

<ipython-input-19-36fd205b194c>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


**Fontos megjegyezni, amikor a tokenizer-t átadod, mint ahogy mi is tettük itt, a Trainer által használt alapértelmezett adat betöltő (data_collator) egy DataCollatorWithPadding lesz, mint amilyet az előbb definiáltunk, tehát kihagyhatod a  data_collator=data_collator sort ebből a hívásból.**

A modell finomhangolásához adathalmazunkon egyszerűen hívjuk a Trainer train() metódusát:




In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: karsarobert (karsar) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
500,0.572400
1000,0.438000


TrainOutput(global_step=1377, training_loss=0.46233711990655635, metrics={'train_runtime': 8469.9209, 'train_samples_per_second': 1.299, 'train_steps_per_second': 0.163, 'total_flos': 405114969714960.0, 'train_loss': 0.46233711990655635, 'epoch': 3.0})

Ez elindítja a finomhangolást (aminek GPU-n pár percet kell igénybe vennie) és a tréning veszteségét 500-anként kiírja. Azonban ez nem fogja megmondani, hogy a modellünk milyen jól (vagy rosszul) teljesít. Ennek oka:

*  Nem állítottuk be a Trainer-t, hogy kiértékelést végezzen tréning közben. Ehhez az evaluation_strategy-t vagy "steps"-re (kiértékelés valahány eval_steps lépésenként) vagy "epoch"-ra kell állítani (kiértékelés minden korszak végén).
* Nem adtunk át a Trainer-nek egy compute_metrics() függvényt, hogy mérőszámokat számítson a kiértékelés során (különben a kiértékelés csak a veszteséget írta volna ki, ami önmagában nem túl informatív).

**Kiértékelés (Evaluation)**

Nézzük meg, hogyan építhetünk egy hasznos compute_metrics() függvényt és alkalmazhatjuk következő tréning során. A függvénynek fel kell vennie egy EvalPrediction objektumot (aminek egy predictions és egy label_ids mezője van) és egy szótárat kell adnia eredményül, amely stringeket lebegőpontos értékekre képez le (a stringek a mérőszámok nevei, a floa-ok pedig az értékeik). A modell jóslataihoz (prediction) a Trainer.predict() parancsot használhatjuk:

```
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)
```

```
(408, 2) (408,)
```

A `predict()` metódus kimenete egy másik, elnevezett tuple három mezővel: `predictions`, `label_ids` és `metrics`. A `metrics` mező csak az átadott adathalmazon bekövetkezett veszteséget, valamint néhány időzítésre vonatkozó metrikát (a predikcióhoz szükséges teljes időt és az átlagos időt) fogja tartalmazni. Miután elkészítjük a `compute_metrics()` függvényünket, és átadjuk a `Trainer`-nek, ez a mező a `compute_metrics()` által visszaadott metrikákat is tartalmazni fogja.

Amint láthatod, a `predictions` egy kétdimenziós tömb, amelynek alakja 408 x 2 (a 408 az általunk használt adathalmazban szereplő elemek száma). Ezek a logitek az általunk a `predict()` függvénynek átadott adathalmaz minden elemére (ahogy az előző fejezetben láttad, minden Transformer modell logiteket ad vissza). Ahhoz, hogy olyan predikciókká alakítsuk át őket, amelyeket a címkéinkkel össze tudunk hasonlítani, a második tengelyen a maximális értékkel rendelkező indexet kell figyelembe vennünk:


In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

Most már össze tudjuk hasonlítani ezeket a predeket a címkékkel. A compute_metric() függvényünk felépítéséhez a 🤗 Evaluate könyvtár metrikáira támaszkodunk. Az MRPC-adatkészlethez kapcsolódó metrikákat ugyanolyan egyszerűen betölthetjük, mint ahogy az adatkészletet betöltöttük, ezúttal az evaluate.load() függvénnyel. A visszaküldött objektum rendelkezik egy compute() metódussal, amellyel elvégezhetjük a metrika kiszámítását:

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8406862745098039, 'f1': 0.8896434634974533}

A pontos eredmények változhatnak, mivel a modellfej véletlenszerű inicializálása megváltoztathatja az elért mérőszámokat. Itt láthatjuk, hogy a modellünk 85,78%-os pontosságot ért el a validációs halmazon, és 89,97-es F1-pontszámot. Ez az a két metrika, amelyet a GLUE benchmark MRPC adathalmazon elért eredmények értékeléséhez használtunk. A BERT-dokumentumban található táblázat 88,9 F1-pontszámot jelentett az alapmodellre. Ez a modell uncased volt, míg mi jelenleg a cased modellt használjuk, ami megmagyarázza a jobb eredményt.

Mindent összetekerve megkapjuk a compute_metrics() függvényünket:

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-25-415be425bb02>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.394416,0.855392,0.897747


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.394416,0.855392,0.897747
2,0.540200,0.427499,0.850490,0.896785
3,0.335500,0.620925,0.843137,0.891156


TrainOutput(global_step=1377, training_loss=0.37138249792737554, metrics={'train_runtime': 9046.4491, 'train_samples_per_second': 1.216, 'train_steps_per_second': 0.152, 'total_flos': 405114969714960.0, 'train_loss': 0.37138249792737554, 'epoch': 3.0})